# Esperimento 3 - Deep Learning


---

**Esperimento:** Completare <a href="https://www.cs.toronto.edu/~hinton/FFA13.pdf">il nuovo algoritmo Forward-Forward</a>, presentato alla conferenza NeurIPS 2022.

In [1]:
import torch
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
p_goodness = 3                              # Esponente della somma utilizzata nella funzione di goodness. E.g.: 1 indica la somma semplice, 2 indica la somma di quadrati etc.
p_norm = 1.5                                # Norma di grado p. https://en.wikipedia.org/wiki/Norm_(mathematics)#p-norm
epochs = 600
batches = 2                                 # Numero di batches per evitare di esaurire la memoria, impostare a 1 per full batch training. La accuracy sull'insieme di training è solo su una batch.
function = torch.nn.SiLU()                  # Funzione di attivazione utilizzata nell'architettura.
neurons = [784,60,60]                       # Neuroni dell'architettura

#from google.colab import drive             # Nel caso si voglia eseguire il codice su Google Colab, togliere i commenti a questi import.
#drive.mount('/content/drive')
#%cd drive/MyDrive/DeepLearning2022

Definiamo una goodness function da utilizzare e la funzione per fare l'embedding dei label nell'input:<br>
References: https://github.com/mohammadpz/pytorch_forward_forward

In [2]:
def goodness_function(x:torch.Tensor, p:int=p_goodness):
    goodness = x.pow(p).mean(1)
    return goodness
    
def label_images(images, labels):
    #Embedding del label nelle immagini.
    x_ = images.clone()
    x_[:, :10] *= 0.0
    x_[range(images.shape[0]), labels] = images.max()
    return x_

def normalize(x:torch.Tensor, p:float=p_norm):
    #Normalizzazione p per layer.
    return x / (x.norm(p, 1, keepdim=True) + 0.0001)


In [3]:
class ReLULayer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = torch.nn.Linear(in_features, out_features)
        self.relu = function
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.03)
        self.threshold = 2.0
        self.num_epochs = epochs*batches

    def forward(self, x):
        x_direction = normalize(x)
        return self.relu(self.linear(x_direction))

    def train(self, x_pos, x_neg):
        for i in range(self.num_epochs):
            positive_goodness = goodness_function(self.forward(x_pos))
            negative_goodness = goodness_function(self.forward(x_neg))
            l = torch.log(1 + torch.exp(torch.cat([
                -positive_goodness + self.threshold,
                negative_goodness - self.threshold]))).mean()
            self.optimizer.zero_grad()
            l.backward()
            self.optimizer.step()
        return self.forward(x_pos).detach(), self.forward(x_neg).detach()

In [4]:
class Net(torch.nn.Module):

    def __init__(self, dimensions):
        super().__init__()
        self.layers = torch.nn.ModuleList([ReLULayer(dimensions[i], dimensions[i + 1]) for i in range(len(dimensions)-1)])
    def predict(self, x):
        goodness_per_label = []
        for label in range(10):
            x_lab = label_images(x, label)
            goodness = []
            for i, layer in enumerate(self.layers):
                x_lab = layer(x_lab)
                if i>0:
                    goodness.append(
                        goodness_function(x_lab)
                    )
            goodness_per_label.append(sum(goodness).unsqueeze(1))
        goodness_per_label = torch.cat(goodness_per_label, 1)
        return torch.argmax(goodness_per_label, dim=1)
    
    def train(self, x_pos, x_neg):
        for layer in self.layers:
            x_pos, x_neg = layer.train(x_pos, x_neg)

In [5]:
torch.manual_seed(0)

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.1307,), (0.3081,)), torchvision.transforms.Lambda(torch.flatten)])

trainset = torchvision.datasets.KMNIST('./data/', transform=transform,  train=True, download=True)                  # La maggior parte dei test sono stati eseguiti sul dataset KMNIST, in quanto più difficile da imparare di MNIST.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=(int(60000/batches)), shuffle=True)

testset = torchvision.datasets.KMNIST('./data/', transform=transform, train=False, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=10000, shuffle=False)

In [6]:
net = Net(neurons).to(device)
x, y = next(iter(trainloader))
x=x.to(device)
y=y.to(device)
x_pos = label_images(x, y)
rnd = torch.randperm(x.size(0))

x_neg = label_images(x, y[rnd])
net.train(x_pos, x_neg)

print('Train accuracy:', net.predict(x).eq(y).float().mean().item())

x_te, y_te = next(iter(testloader))
x_te=x_te.to(device)
y_te=y_te.to(device)

print('Test accuracy:', net.predict(x_te).eq(y_te).float().mean().item())

torch.cuda.empty_cache()

Train accuracy: 0.8967999815940857
Test accuracy: 0.7622999548912048


---

### Analisi delle funzioni di attivazione: "What is the best activation function to use?"

Sono state esaminate varie funzioni di attivazione inizializzate con vari iperparametri diversi, e sono emersi i seguenti risultati sui dataset MNIST/KMNIST:
- La funzione ReLU è la funzione che offre le performance migliori, ed è la più portata per questi dataset.
- Le funzioni limitate come sigmoid e tanh tendono a non performare bene a causa del valore di threshold.
- Altre funzioni illimitate che posseggono parte negativa come SiLU o ELU posseggono maggiore capacità di esplorazione, in quanto non sono limitate all'ortante positivo, ma tendono a performare peggio rispetto a ReLU con lo scalare della dimensione dell'architettura.

Per quanto riguarda la normalizzazione, è stato osservato che altri tipi di norme sono capaci di performare alla pari, o meglio, della norma 2. Un esempio è la norma 1.8, che porta a un miglioramento sull'insieme di test medio del ~3% sul dataset KMNIST.
È quindi valido supporre l'esistenza di norme e/o di funzioni di attivazione migliori capaci di raggiungere una performance migliore con l'algoritmo Forward-Forward.

---

### Analisi della funzione di goodness: "What is the best goodness function to use?"

Esaminando la funzione di goodness, è stata valutata la possibilità di avere somme di valori non strettamente positivi quali somma di cubi e somma semplice.<br>
Questo, teoricamente, consentirebbe alla funzione di goodness di avere valori negativi (per funzioni di attivazione diverse da ReLU), influenzando il valore di goodness negativo e consentendo all'architettura di aumentare l'esplorazione.<br>
È stato osservato che il valore ottimale di norma dipende dal tipo di somma effettuato (generalmente in maniera inversamente proporzionale rispetto al grado), e che la somma di cubi è un'ottima funzione di goodness per quanto riguarda le funzioni illimitate con valori negativi (e.g. SiLU), con un'accuracy media che raggiunge l'80% sull'insieme di test per quanto riguarda il dataset KMNIST per l'architettura considerata, superando persino la classica somma di quadrati con la funzione ReLU.<br>
Per quanto riguarda la somma semplice sono stati ottenuti risultati accettabili (~65% accuratezza sull'insieme di test) ma non al livello della classica funzione di goodness di somma di quadrati.

È necessario evidenziare come l'architettura presentata (784,500,500) sia decisamente esagerata e tenda decisamente all'overfitting. Un'architettura molto più piccola (784,60,60), fermata al numero adeguato di epochs, è capace di performare quasi alla pari dell'architettura presentata (~76% invece di ~80%) e presenta ancora segni di overfitting. È quindi evidente che è possibile espandere e studiare ulteriormente quest'architettura per raggiungere risultati ancora più soddisfacenti.